In [1]:
#0
#datapreproc - moreallfiles
#AAI-540 Group 3 FP

In [2]:
import pandas as pd
import numpy as np 

In [3]:
simplebench = pd.read_csv('simplebench_external.csv')

In [4]:
simplebench.head(5)

,Model version,Score (AVG@5),Release date,Organization,Country,Training compute (FLOP),Training compute notes,Source,Source link (site from table),Notes,id
0,gemini-3-pro-preview,0.764,2025-11-18,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,Almost certainly >1e25,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recyUT7yjrhcHvcWI
1,gemini-2.5-pro-preview-06-05,0.624,2025-06-05,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recViPEOMOrSB6VKK
2,claude-opus-4-5-20251101,0.620,2025-11-24,Anthropic,United States of America,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,NaN,NaN,reclKBQ393HPkOMed
3,gpt-5-pro-2025-10-06_high,0.616,2025-10-07,OpenAI,United States of America,NaN,NaN,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recbQ9dn731JxU1au
4,gemini-3-flash-preview,0.611,2025-12-17,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,NaN,SimpleBench Leaderboard,NaN,NaN,rectQAsXBWhS8ne4o


In [5]:
simplebench.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Model version                  52 non-null     object 
 1   Score (AVG@5)                  55 non-null     float64
 2   Release date                   51 non-null     object 
 3   Organization                   52 non-null     object 
 4   Country                        52 non-null     object 
 5   Training compute (FLOP)        21 non-null     float64
 6   Training compute notes         38 non-null     object 
 7   Source                         55 non-null     object 
 8   Source link (site from table)  50 non-null     object 
 9   Notes                          5 non-null      object 
 10  id                             55 non-null     object 
dtypes: float64(2), object(9)
memory usage: 4.9+ KB


In [6]:
original_file = 'simplebench_external.csv'
simplebench.to_csv(original_file, index=False)

In [7]:
!aws s3 cp simplebench_external.csv s3://sagemaker-us-east-1-342942554664/simplebench_external.csv

upload: ./simplebench_external.csv to s3://sagemaker-us-east-1-342942554664/simplebench_external.csv


In [47]:
#10
#athena db config - simplebench
#AAI-540 Group 3 FP

In [8]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [9]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [10]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_10720/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [11]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_10720/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [12]:
s3_data_path = 's3://sagemaker-us-east-1-342942554664/simplebench_external.csv'
s3_table_path = 's3://sagemaker-us-east-1-342942554664/table7/'

In [13]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-342942554664/simplebench_external.csv to s3://sagemaker-us-east-1-342942554664/table7/simplebench_external.csv


In [14]:
#table7 - simplebench.csv
table_name_csv = 'simplebenchmark'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
dataexplore = pd.read_csv('simplebench_external.csv')
dataexplore.head()

,Model version,Score (AVG@5),Release date,Organization,Country,Training compute (FLOP),Training compute notes,Source,Source link (site from table),Notes,id
0,gemini-3-pro-preview,0.764,2025-11-18,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,Almost certainly >1e25,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recyUT7yjrhcHvcWI
1,gemini-2.5-pro-preview-06-05,0.624,2025-06-05,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recViPEOMOrSB6VKK
2,claude-opus-4-5-20251101,0.620,2025-11-24,Anthropic,United States of America,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,NaN,NaN,reclKBQ393HPkOMed
3,gpt-5-pro-2025-10-06_high,0.616,2025-10-07,OpenAI,United States of America,NaN,NaN,SimpleBench Leaderboard,https://simple-bench.com/,NaN,recbQ9dn731JxU1au
4,gemini-3-flash-preview,0.611,2025-12-17,Google DeepMind,United Kingdom of Great Britain and Northern I...,NaN,NaN,SimpleBench Leaderboard,NaN,NaN,rectQAsXBWhS8ne4o


In [15]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Model version                  52 non-null     object 
 1   Score (AVG@5)                  55 non-null     float64
 2   Release date                   51 non-null     object 
 3   Organization                   52 non-null     object 
 4   Country                        52 non-null     object 
 5   Training compute (FLOP)        21 non-null     float64
 6   Training compute notes         38 non-null     object 
 7   Source                         55 non-null     object 
 8   Source link (site from table)  50 non-null     object 
 9   Notes                          5 non-null      object 
 10  id                             55 non-null     object 
dtypes: float64(2), object(9)
memory usage: 4.9+ KB


In [16]:
!aws s3 cp s3://sagemaker-us-east-1-342942554664/simplebench_external.csv \
           s3://sagemaker-us-east-1-342942554664/table7/

copy: s3://sagemaker-us-east-1-342942554664/simplebench_external.csv to s3://sagemaker-us-east-1-342942554664/table7/simplebench_external.csv


In [17]:
!aws s3 ls s3://sagemaker-us-east-1-342942554664/table7/

2026-02-12 10:10:53      18538 simplebench_external.csv


In [18]:
#sql table creation
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         modelversion STRING,
         score FLOAT,
         release_date STRING,
         organization STRING,
         country STRING,
         trainingcomputer FLOAT,
         trainingnotes STRING,
         source STRING,
         sourcelink STRING,
         notes STRING,
         modelid STRING
) ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_table_path
)

In [19]:
pd.read_sql(statement, conn)

/tmp/ipykernel_10720/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [21]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(8)

/tmp/ipykernel_10720/3158068780.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll
1,lifearchitect
2,lifearchitectmodels
3,llmachievements
4,llmleader
5,llmpricing
6,overviewmodel
7,simplebenchmark


In [22]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.simplebenchmark LIMIT 10


/tmp/ipykernel_10720/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,modelversion,score,release_date,organization,country,trainingcomputer,trainingnotes,source,sourcelink,notes,modelid
0,gemini-3-pro-preview,0.764,2025-11-18,Google DeepMind,"""United Kingdom of Great Britain and Northern ...",NaN,,Almost certainly >1e25,SimpleBench Leaderboard,https://simple-bench.com/,
1,gemini-2.5-pro-preview-06-05,0.624,2025-06-05,Google DeepMind,"""United Kingdom of Great Britain and Northern ...",NaN,,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,https://simple-bench.com/,
2,claude-opus-4-5-20251101,0.620,2025-11-24,Anthropic,United States of America,NaN,Flagship model from a leading developer in mid...,SimpleBench Leaderboard,,,reclKBQ393HPkOMed
3,gpt-5-pro-2025-10-06_high,0.616,2025-10-07,OpenAI,United States of America,NaN,,SimpleBench Leaderboard,https://simple-bench.com/,,recbQ9dn731JxU1au
4,gemini-3-flash-preview,0.611,2025-12-17,Google DeepMind,"""United Kingdom of Great Britain and Northern ...",NaN,,,SimpleBench Leaderboard,,
